#  <center>Project 1 - Fully convolutional neural network</center>

The objective of this project is to test different architectures to compare two digits visible in a
two-channel image. It aims at showing in particular the impact of weight sharing, and of the use of an
auxiliary loss to help the training of the main objective.
It should be implemented with PyTorch only code, in particular without using other external libraries
such as scikit-learn or numpy.


In [2]:
import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F
import dlc_practical_prologue as prologue

# Loading the data

We generate the data sets to using with the function generate_pair_sets(N) defined in the file
dlc_practical_prologue.py. This function returns six tensors:


| Name          | Tensor dimension | Type     | Content                                    |
|---------------|------------------|----------|--------------------------------------------|
| train_input   | N × 2 × 14 × 14  | float32  | Images                                     |
| train_target  | N                | int64    | Class to predict ∈ {0, 1}                  |
| train_classes | N × 2            | int64    | Classes of the two digits ∈ {0, . . . , 9} |
| test_input    | N × 2 × 14 × 14  | float32  | Images                                     |
| test_target   | N                | int64    | Class to predict ∈ {0, 1}                  |
| test_classes  | N × 2            | int64    | Classes of the two digits ∈ {0, . . . , 9} |

In [3]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)

In [ ]:
mu,std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)

mu,std = test_input.mean(), test_input.std()
test_input.sub_(mu).div_(std)

# Defining the Model

In [17]:

######################################################################

class ResNetBlock(nn.Module):
    def __init__(self, nb_channels, kernel_size,skip_connections,batch_normalization):
        super(ResNetBlock, self).__init__()

        self.conv1 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn1 = nn.BatchNorm2d(nb_channels)

        self.conv2 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn2 = nn.BatchNorm2d(nb_channels)
        self.skip_connections = skip_connections
        self.batch_normalization= batch_normalization

    def forward(self, x):
        y = self.conv1(x)
        if self.batch_normalization: y = self.bn1(y)
        y = F.relu(y)
        y = self.conv2(y)
        if self.batch_normalization: y = self.bn2(y)
        if self.skip_connections : y = y + x
        y = F.relu(y)

        return y


######################################################################

class ResNet(nn.Module):
    def __init__(self, nb_residual_blocks, nb_channels,
                 kernel_size = 3, nb_classes = 10,skip_connections=True,batch_normalization=True):
        super(ResNet, self).__init__()

        self.conv = nn.Conv2d(2, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn = nn.BatchNorm2d(nb_channels)
        self.skip_connections = skip_connections
        self.batch_normalization= batch_normalization        

        self.resnet_blocks = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size,skip_connections,batch_normalization)
              for _ in range(nb_residual_blocks))
        )

        self.fc = nn.Linear(nb_channels, nb_classes)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = self.resnet_blocks(x)
        x = F.avg_pool2d(x, 14).view(x.size(0), -1)
        x = self.fc(x)
        return x



# Training the model

CNN with 2 convolutional layers and 2 fully connected layers 

In [37]:
batch_size = 100
n_epochs = 25
def train_model(model,train_input,train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.2)
    for e in range(0,n_epochs):
        for input, targets in zip(train_input.split(batch_size),train_target.split(batch_size)):
            output = model(input)
            loss = criterion(output,targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if(e%5 ==0):
            print('epoch : ',e,' loss : ',loss.item())

In [38]:

model = ResNet(nb_residual_blocks = 15, nb_channels = 10,
               kernel_size = 3, nb_classes = 2)
train_model(model,train_input,train_target)

epoch :  0  loss :  0.7117371559143066
epoch :  5  loss :  0.41849586367607117
epoch :  10  loss :  0.4240015745162964
epoch :  15  loss :  0.31182900071144104
epoch :  20  loss :  0.43093544244766235


# Testing

In [ ]:
def compute_nb_errors(model,data_input,data_target):
    nb_errors = 0
    for input,targets in zip(data_input.split(batch_size),data_target.split(batch_size)):
        output = model(input)
        _,predicted_classes = torch.max(output,1)
        for i in range(0,output.size(0)):
            if(predicted_classes[i]!=targets[i]):
                nb_errors = nb_errors+1
                
    return nb_errors

In [ ]:
print('train_error {:.02f}% test_error {:.02f}%'.format(
    compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100,
    compute_nb_errors(model, test_input, test_target) / test_input.size(0) * 100))
